In [3]:
import cv2
import numpy as np
import winsound

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Taking video feed
## EDIT THE LOCATION OF THE VIDEO
video = cv2.VideoCapture(r"C:\Users\matth\0_FYP\Testing\falling_test.mp4")

# Create Background Subtractor kernel
fgbg = cv2.createBackgroundSubtractorKNN(128, cv2.THRESH_BINARY, 1)
h_list = []
framenum = 0

while True:
    framenum += 1
    # Obtain frame
    ret, frame = video.read()

    # Check if the frame is empty
    if not ret or frame is None:
        print("End of video. Exiting...")
        break

    # Skip frames for faster processing
    if framenum % 5 != 0:
        continue

    height, width, channels = frame.shape

    # Detecting people using YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    fall_detected = False

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.6 and class_id == 0:  # Class ID for person is 0
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                # Check if the height has decreased significantly, indicating a fall
                if len(h_list) > 0 and h < 0.8 * min(h_list):
                    winsound.Beep(2500, 2000)
                    print("Fall Detected")

                    # Change the color of the square and label it as "Fall"
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    cv2.putText(frame, "Fall", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2,
                                cv2.LINE_AA)

                    fall_detected = True
                else:
                    # Draw bounding box
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(frame, "Person", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2,
                                cv2.LINE_AA)

                h_list.append(h)

    # Reset fall detection after a certain period of time
    if fall_detected and framenum % 30 == 0:  # Assuming 30 frames per second
        fall_detected = False

    # The feed showing the bounding box
    cv2.imshow('Security feed', frame)

    k = cv2.waitKey(1) & 0xff
    if k == 27:
        break

video.release()
cv2.destroyAllWindows()

testing
